<a href="https://colab.research.google.com/github/elizathornton/Fax-handling-with-OCR/blob/main/fax_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install poppler-utils
!pip install Pillow
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pdf2image

import pytesseract
import shutil
import os
import random
import PIL
import glob
import pandas as pd
import numpy as np
import cv2

from pdf2image import convert_from_path, convert_from_bytes
#from IPython.display import display, Image
from PIL import Image


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.14).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# download patient database
worksheet = gc.open('OCR test spreadsheets').worksheet('Fake Patient List')
rows = worksheet.get_all_values()
patdf = pd.DataFrame.from_records(rows)   # patdf is the dataframe of patient names

patdf.columns = patdf.iloc[0]
patdf.drop(patdf.index[0], inplace=True)
patdf.head()

#download keywords for type database
worksheet = gc.open('OCR test spreadsheets').worksheet('Keywords for type')
rows = worksheet.get_all_values()
typedf = pd.DataFrame.from_records(rows)   # typedf is the dataframe of PDF types
typedf.columns = ['Type','Phrase']

#download keywords for type database
worksheet = gc.open('OCR test spreadsheets').worksheet('Keywords for common descriptor')
rows = worksheet.get_all_values()
desdf = pd.DataFrame.from_records(rows)   # desdf is the dataframe of common descriptors for PDF types
desdf.columns = ['Descriptor','Phrase']


In [19]:

print(desdf.iloc[1][0])



Labcorp


In [6]:
desdf.head()

,Descriptor,Phrase
0,Common description,Key Phrase
1,PAIEB Form,DEPARTMENT OF HUMAN SERVICES Physician Certifi...
2,Labcorp,FROM:LABCORP


In [19]:
#  Step 1 - select a pdf files
# Step 2 - convert each of the pages to jpg
# NOT WORKING - pillow isn't the correct version, or  maybe I need to build a poppler path? - come back to this. 

pdfs = "/content/drive/My Drive/VPA projects/OCR Fax/death.pdf"
#OCR Fax/PDFs of FAKE faxes/FAKE PATIENT paieb.pdf"

pages = convert_from_path(pdfs, poppler_path='/usr/bin/')


data = ''
for page in pages:
  data = data + pytesseract.image_to_string(page)  
  
print(data)




 

 

 

 

 

To: Wk 2a thant
F A x Fax number: Q \S-DDl)~- Sle.

Molden Funeral Chapel & From: Ca,
Cremation Service Fax number: 215-788-4210

133 Otter St. |

Bristol, PA 19007 Date: L4 / \\ a x

215-788-6612 pif

moldenfuneralchapel.com Regarding: Medical Certificatinn Worksheet
[Subject]

 

 

/ : ed
/ | Phone number for follow-up:
a 215-788-6612

 

 

 

 

 

URGENT REPLY REQUESTED |

« Complete the attached Medical Certification Worksheet.

Do not leave any items blank, including items 27 through
38.

e Did you enter the chain of events that directly caused the death? Remember to enter the
approximate interval: onset to death for each condition listed.

« Are the entries legible?

« Please sign and date the form and be sure to include your name, address and license
number typed or handwritten legibly.

e All information from this worksheet will be entered into the electronic system.

e Is the certifier a physician, CRNP, or PA-C? Remember, RN’s cannot certify a death in PA.



In [25]:
patdf.head(10)

,First Name,Last Name,DOB,Provider
1,Elizabeth,Bennet,1/1/1930,ET
2,George,Wickham,2/2/1942,MA
3,Fitzwilliam,Darcy,3/17/1925,TT
4,Lydia,Bennet,4/1/1960,ET
5,Charlotte,Lucas,7/8/1971,MA
6,Catherine,De-Bourgh,1/2/1934,TT
7,Charles,Bingley,5/4/1950,ET


In [16]:

# create lists as global variables
# these are all lists from the ocr spreadsheets that we can use inside our function to search for patient and 
#type in the pdfs

lastnames = patdf['Last Name'].tolist()
firstnames = patdf['First Name'].tolist()
doblist = patdf['DOB'].tolist()
provider = patdf['Provider'].tolist()
numpatients = len(lastnames)

typelist =  typedf['Phrase'].tolist()
deslist =  desdf['Phrase'].tolist()

In [17]:
print(doblist)

['1/1/1930', '2/2/1942', '3/17/1925', '4/1/1960', '7/8/1971', '1/2/1934', '5/4/1950']


In [18]:
# search for patient name
#search for a last name.  Some patients have common last names and there will be multiple possible patients. 


possiblepatients = []
i = 0
while i< numpatients:
  last = lastnames[i]
  if last and (last.casefold() in data.casefold()):  # if the descriptive phrase is not null and is found in the data
    possiblepatients.append(i) # add one b/c we are oging to use it on the dataframe, not the list
    print('Found last name:'+last)
  print(i)
  i+=1
    

print(possiblepatients)

#if you find a last name look for a first name nearby
newpossiblepatients = []
for i in possiblepatients:
  first = firstnames[i]
  if first and (first.casefold() in data.casefold()):  # if the first name is not null and is found in the data
    newpossiblepatients.append(i)
    print('Found First name:'+first)

print(newpossiblepatients)

j = -1  # j is the index of the patient in the lists
if len(newpossiblepatients) == 1:
  j = newpossiblepatients[0] # j is the index for the list of the single patient found on our search
elif len(newpossiblepatients) ==0:
  j=-1
  #do nothing - j is still -1 which indicates we didn't identify the patient in the PDF
elif len(newpossiblepatients) > 1:  #if there are still 2 possible patients, look for a DOB
  newnewpossiblepatients = []  
  for i in newpossiblepatients:
    dob = doblist[i]
    if dob and (dob in data):  # if the dob is not null and is found in the data
      newnewpossiblepatients.append(i)

  if len(newnewpossiblepatients) ==1 :  # if there is only one patient left, we have identified the patient
    j = newnewpossiblepatients[0]
   #no need to do an else statement here b/c if there are still 2 or more  possible patients or NONE then tag that we couldn't name the file

if j > -1:
  filename = provider[j]+' '+firstnames[j]+' '+lastnames[j]+' DOB_'+doblist[j].replace('/', '_')
  print(filename)


# search for type
pdftype = ''   # The type of the PDF
for t in typelist:
  if t and data.find(t) != -1:  # if the phrase is not null and is found in the data
    pdftype = typedf.iloc[typelist.index(t)][0]
    filename = filename+' '+pdftype
    break
print(pdftype)


# search for common descriptors
descriptor = ''   # The common descriptors for the PDF
for des in deslist:
  if des and data.find(des) != -1:  # if the descriptive phrase is not null and is found in the data
    descriptor = desdf.iloc[deslist.index(des)][0]
    filename = filename+' '+descriptor
    break
print(filename)




Found last name:Bennet
0
1
2
Found last name:Bennet
3
4
5
6
[0, 3]
Found First name:Elizabeth
[0]
ET Elizabeth Bennet DOB_1_1_1930
SNF_Waiver
ET Elizabeth Bennet DOB_1_1_1930 SNF_Waiver PAIEB Form
